# Project4 - Host program

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

from pynq import Overlay
import pynq.lib.dma
from pynq import Xlnk
from pynq import MMIO

o1 = Overlay('/home/xilinx/jupyter_notebooks/OFDM/OFDM.bit')
# Download your bitstream to FPGA
t_before_bitstream = time.time()
o1.download()
t_after_bitstream = time.time()

print(t_after_bitstream - t_before_bitstream, 'seconds to program bitstream')

In [ ]:
xlnk = Xlnk() # Contiguous Memory Allocator (CMA)

In [ ]:
length = 35000

# Open input/output files
fI = open('input.dat','r')

fG = open('out.gold.dat', 'r')

# Allocate regular numpy arrays to store input and output
inp = np.empty([length,], dtype=np.float32)

golden = np.empty([length,], dtype=np.float32)

# Store data into arrays
for i in range (0, length):
    golden[i] = [np.float32(x) for x in next(fG).split()]
    inp[i] = np.float32(next(fI))
    

In [ ]:
plt.plot(golden)

## Complete the following block

In [ ]:
# Allocate CMA array for DMA
xlnk = Xlnk()
# Copy regular numpy arrays to CMA arrays
dma = o1.axi_dma_0
# Write length using MMIO (we got the address from Vivado)
OFDM_ip = MMIO(0x43c00000,0x10000)

size = 35000
in_buffer = xlnk.cma_array(shape=(size,), dtype=np.float32)
out_buffer = xlnk.cma_array(shape=(size,), dtype=np.float32)

fI = open('input.dat','r')
fG = open('out.gold.dat', 'r')
for i in range (0, size):
    in_buffer[i] = np.float32(next(fI))
    
OFDM_ip.write(0x10,size)
t_start = time.time()
# Begin data transfer from/to DMA
dma.sendchannel.transfer(in_buffer)

dma.recvchannel.transfer(out_buffer)

dma.sendchannel.wait()

dma.recvchannel.wait()

t_stop = time.time()

# Free the CMA buffers
in_buffer.close()

out_buffer.close()


print(t_after_bitstream - t_before_bitstream, 'seconds to execute on hardware')

In [ ]:
plt.plot(out_buffer)